# Theano Logistic Regression Tutorial

## Imports

In [1]:
import numpy
import theano
import theano.tensor as T

## Setup

In [2]:
N = 400
feats = 784
D = (numpy.random.randn(N, feats), numpy.random.randint(size=N, low=0, high=2))
training_steps = 10000

## Declare Theano symbolic variables

In [3]:
x = T.matrix("x")
y = T.vector("y")
w = theano.shared(numpy.random.randn(feats), name="w")
b = theano.shared(0., name="b")

## Construct Theano expression graph

In [4]:
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b)) # Probability that target = 1
prediction = p_1 > 0.5 # The prediction thresholded
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # Cross-entropy loss function
cost = xent.mean() + 0.01 * (w ** 2).sum() # The cost to minimize

## Compute the gradient of the cost

In [5]:
# We shall return to this in a following section of this tutorial
gw, gb = T.grad(cost, [w, b])

## Compile

In [6]:
train = theano.function(
          inputs=[x,y],
          outputs=[prediction, xent],
          updates=((w, w - 0.1 * gw), (b, b - 0.1 * gb)))
predict = theano.function(inputs=[x], outputs=prediction)

## Train

In [7]:
for i in range(training_steps):
    pred, err = train(D[0], D[1])

print("Final model:")
print(w.get_value())
print(b.get_value())
print("target values for D:")
print(D[1])
print("prediction on D:")
print(predict(D[0]))

Final model:
[-0.01518821 -0.01415944  0.00434382 -0.15363162 -0.18003992 -0.01253708
  0.03651842  0.03146085 -0.1093249   0.07179806  0.0323893  -0.05965273
  0.0911283   0.04459685  0.18428166 -0.08798417 -0.104639    0.00862909
  0.00036649 -0.22661076 -0.26454289  0.03635827  0.14378412  0.1579776
  0.07099326 -0.05328881 -0.01214565  0.06027384  0.09251733 -0.13439394
 -0.01135125 -0.10781403  0.0085375  -0.03371679  0.00039016  0.0022424
 -0.03139585  0.07057585  0.13070885 -0.11131814  0.0291172   0.01575916
 -0.04829049  0.1003439   0.09820573 -0.01081475  0.0602273   0.13837639
  0.13733547  0.02449358  0.04701734 -0.11468713 -0.0281512   0.04503989
 -0.05598592  0.01593866  0.06985218  0.14745925 -0.06645025  0.08512957
 -0.04630519  0.03712248 -0.09766912 -0.16983958  0.00213887 -0.15186493
 -0.06347896  0.01726196 -0.34702372 -0.03402589  0.19021033 -0.07343035
  0.02953966  0.07030712 -0.028319   -0.05501454  0.06830752 -0.03099222
 -0.08475477 -0.02768515 -0.10258879  0.

In [9]:
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

[Elemwise{exp,no_inplace}(<TensorType(float64, vector)>)]
Looping 1000 times took 1.401809 seconds
Result is [ 1.23178032  1.61879341  1.52278065 ...,  2.20771815  2.29967753
  1.62323285]
Used the cpu
